In [1]:
import json
import pandas as pd
import warnings
import json
from tqdm import tqdm
import re

# dataset_for_vector불러오기

In [89]:
DATA_DIR = "../data/"
data_file = "dataset_for_vector.csv"
dataset = pd.read_csv(DATA_DIR + data_file, encoding='UTF-8')
dataset.head()

,title,sentence,extractive
0,"논 타작물 재배, 2월 말까지 신청하세요","['ha당 조사료 400만원…작물별 차등 지원', '이성훈 sinawi@hanmai...","[2, 3, 10]"
1,"여수시, 컬러빌리지 마무리...색채와 빛도시 완성","['8억 투입, 고소천사벽화·자산마을에 색채 입혀', '이성훈 sinawi@hanm...","[2, 4, 11]"
2,새해 정기 받고 올해는 반드시 일내자!,"['전남드래곤즈 해맞이 다짐…선수 영입 활발', '이성훈 sinawi@hanmail...","[3, 5, 7]"
3,"농업인 역량 강화, 새해 실용교육 실시","['11~24일, 매실·감·참다래 등 지역특화작목', '이성훈 sinawi@hanm...","[2, 3, 4]"
4,타이완 크루즈관광객 4천여명전남온다,"['홍콩 크루즈선사아쿠아리우스 4, 6월 여수항 입항', '이성훈 sinawi@ha...","[3, 7, 4]"


In [90]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243977 entries, 0 to 243976
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   title       243977 non-null  object
 1   sentence    243977 non-null  object
 2   extractive  243977 non-null  object
dtypes: object(3)
memory usage: 5.6+ MB


In [91]:
dataset.isna().sum()

title         0
sentence      0
extractive    0
dtype: int64

# 데이터 전처리

### 'sentence'와 'extractive'컬럼 리스트 타입으로 변경
데이터 불러오면서 string으로 바뀜...

In [92]:
dataset['sentence'][0][0]

'['

In [93]:
dataset['extractive'][0][0]

'['

In [94]:
# sentence와 extractive 컬럼의 string을 리스트로 변환
import ast

dataset['sentence'] = dataset['sentence'].apply(ast.literal_eval)
dataset['extractive'] = dataset['extractive'].apply(ast.literal_eval)

In [96]:
print(dataset['sentence'][0][0])
print(dataset['extractive'][0][0])

ha당 조사료 400만원…작물별 차등 지원
2


### ex_sentence 컬럼 추가
extracitve에 해당하는 문장 추출

In [98]:
# extractive에 해당하는 문장 뽑는 함수 구현
def extract_sentence(row):
    return [row['sentence'][index] for index in row['extractive']]

dataset['ex_sentence'] = dataset.apply(extract_sentence, axis=1)

In [99]:
dataset.head()

,title,sentence,extractive,ex_sentence
0,"논 타작물 재배, 2월 말까지 신청하세요","[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...","[2, 3, 10]",[전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제...
1,"여수시, 컬러빌리지 마무리...색채와 빛도시 완성","[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail...","[2, 4, 11]",[여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛...
2,새해 정기 받고 올해는 반드시 일내자!,"[전남드래곤즈 해맞이 다짐…선수 영입 활발, 이성훈 sinawi@hanmail.ne...","[3, 5, 7]",[임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목표...
3,"농업인 역량 강화, 새해 실용교육 실시","[11~24일, 매실·감·참다래 등 지역특화작목, 이성훈 sinawi@hanmail...","[2, 3, 4]",[광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 105...
4,타이완 크루즈관광객 4천여명전남온다,"[홍콩 크루즈선사아쿠아리우스 4, 6월 여수항 입항, 이성훈 sinawi@hanma...","[3, 7, 4]",[전라남도는 올해 4월과 6월 홍콩 크루즈선사 스타크루즈의 5만톤급아쿠아리우스호가 ...


In [100]:
dataset['ex_sentence'][0][0]

'전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다.'

### id 컬럼 추가
id는 각 기사별로 고유의 값 가질 수 있도록 함

In [ ]:
dataset.reset_index(drop=True)

In [103]:
dataset['id'] = dataset.index + 1

In [104]:
dataset.head()

,title,sentence,extractive,ex_sentence,id
0,"논 타작물 재배, 2월 말까지 신청하세요","[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...","[2, 3, 10]",[전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제...,1
1,"여수시, 컬러빌리지 마무리...색채와 빛도시 완성","[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail...","[2, 4, 11]",[여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛...,2
2,새해 정기 받고 올해는 반드시 일내자!,"[전남드래곤즈 해맞이 다짐…선수 영입 활발, 이성훈 sinawi@hanmail.ne...","[3, 5, 7]",[임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목표...,3
3,"농업인 역량 강화, 새해 실용교육 실시","[11~24일, 매실·감·참다래 등 지역특화작목, 이성훈 sinawi@hanmail...","[2, 3, 4]",[광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 105...,4
4,타이완 크루즈관광객 4천여명전남온다,"[홍콩 크루즈선사아쿠아리우스 4, 6월 여수항 입항, 이성훈 sinawi@hanma...","[3, 7, 4]",[전라남도는 올해 4월과 6월 홍콩 크루즈선사 스타크루즈의 5만톤급아쿠아리우스호가 ...,5


In [105]:
# 컬럼 순서 바꾸기
dataset = dataset[['id', 'title', 'sentence', 'extractive', 'ex_sentence']]

In [106]:
dataset.head()

,id,title,sentence,extractive,ex_sentence
0,1,"논 타작물 재배, 2월 말까지 신청하세요","[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...","[2, 3, 10]",[전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제...
1,2,"여수시, 컬러빌리지 마무리...색채와 빛도시 완성","[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail...","[2, 4, 11]",[여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛...
2,3,새해 정기 받고 올해는 반드시 일내자!,"[전남드래곤즈 해맞이 다짐…선수 영입 활발, 이성훈 sinawi@hanmail.ne...","[3, 5, 7]",[임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목표...
3,4,"농업인 역량 강화, 새해 실용교육 실시","[11~24일, 매실·감·참다래 등 지역특화작목, 이성훈 sinawi@hanmail...","[2, 3, 4]",[광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 105...
4,5,타이완 크루즈관광객 4천여명전남온다,"[홍콩 크루즈선사아쿠아리우스 4, 6월 여수항 입항, 이성훈 sinawi@hanma...","[3, 7, 4]",[전라남도는 올해 4월과 6월 홍콩 크루즈선사 스타크루즈의 5만톤급아쿠아리우스호가 ...


In [107]:
dataset.tail()

,id,title,sentence,extractive,ex_sentence
243972,243973,"당진시, 찾아가는 청소년 노동인권 교육 진행","[내달 12일까지, 노동인권 감수성 높인다, 당진시는 지난 12일 첫 교육을 시작으...","[1, 2, 6]",[당진시는 지난 12일 첫 교육을 시작으로 내달 12일까지 당진지역 청소년들을 대상...
243973,243974,"서산시, 2020년 성인문해교실 신청하세요!","[마을별 10명 이상이면 신청가능, 12월 24일까지 모집, 서산 마을학교 합동졸업...","[5, 4, 8]","[성인문해교실(마을학교)은 학령기 동안 경제적, 사회적 환경 등의 어려움으로 배움의..."
243974,243975,"서산시의회, 제247회 제2차 정례회 개회","[2020년 예산안, 제4회 추경예산안 및 조례안 29건 등 41개 안건 처리 예정...","[1, 3, 4]","[서산시의회는 25일 제1차 본회의를 열고 제247회 제2차 정례회를 개회했다., ..."
243975,243976,"아산시, 온천 재활 헬스케어 힐링산업 본격 추진","[전담기관 설립 추진, 타당성 검토 용역 착수보고회 개최, 아산시(시장 오세현)가 ...","[1, 2, 3]",[아산시(시장 오세현)가 온천수를 활용한 재활 헬스케어 힐링산업에 본격 착수할 예정...
243976,243977,"용인 비상에듀 기숙학원, 대입조기선발반 모집... 12월 15일 개강","[용인비상에듀기숙학원 전경, 입시 전문 재수기숙학원 용인비상에듀기숙학원이 2021학...","[1, 5, 8]",[입시 전문 재수기숙학원 용인비상에듀기숙학원이 2021학년도 수능을 준비하는 고교 ...


# article_table 내보내기

In [108]:
article_dataset = dataset[['id', 'title', 'sentence']]

In [109]:
article_dataset.head()

,id,title,sentence
0,1,"논 타작물 재배, 2월 말까지 신청하세요","[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n..."
1,2,"여수시, 컬러빌리지 마무리...색채와 빛도시 완성","[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail..."
2,3,새해 정기 받고 올해는 반드시 일내자!,"[전남드래곤즈 해맞이 다짐…선수 영입 활발, 이성훈 sinawi@hanmail.ne..."
3,4,"농업인 역량 강화, 새해 실용교육 실시","[11~24일, 매실·감·참다래 등 지역특화작목, 이성훈 sinawi@hanmail..."
4,5,타이완 크루즈관광객 4천여명전남온다,"[홍콩 크루즈선사아쿠아리우스 4, 6월 여수항 입항, 이성훈 sinawi@hanma..."


In [80]:
# sentence를 article로 컬럼명 변경
article_dataset.rename(columns={'sentence':'article'}, inplace=True)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_27108\1667279573.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  article_dataset.rename(columns={'sentence':'article'}, inplace=True)


In [82]:
article_dataset.head()

,id,title,article
0,1,"논 타작물 재배, 2월 말까지 신청하세요","[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n..."
1,2,"여수시, 컬러빌리지 마무리...색채와 빛도시 완성","[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail..."
2,3,새해 정기 받고 올해는 반드시 일내자!,"[전남드래곤즈 해맞이 다짐…선수 영입 활발, 이성훈 sinawi@hanmail.ne..."
3,4,"농업인 역량 강화, 새해 실용교육 실시","[11~24일, 매실·감·참다래 등 지역특화작목, 이성훈 sinawi@hanmail..."
4,5,타이완 크루즈관광객 4천여명전남온다,"[홍콩 크루즈선사아쿠아리우스 4, 6월 여수항 입항, 이성훈 sinawi@hanma..."


In [86]:
article_dataset.tail()

,id,title,article
243972,243973,"당진시, 찾아가는 청소년 노동인권 교육 진행","[내달 12일까지, 노동인권 감수성 높인다, 당진시는 지난 12일 첫 교육을 시작으..."
243973,243974,"서산시, 2020년 성인문해교실 신청하세요!","[마을별 10명 이상이면 신청가능, 12월 24일까지 모집, 서산 마을학교 합동졸업..."
243974,243975,"서산시의회, 제247회 제2차 정례회 개회","[2020년 예산안, 제4회 추경예산안 및 조례안 29건 등 41개 안건 처리 예정..."
243975,243976,"아산시, 온천 재활 헬스케어 힐링산업 본격 추진","[전담기관 설립 추진, 타당성 검토 용역 착수보고회 개최, 아산시(시장 오세현)가 ..."
243976,243977,"용인 비상에듀 기숙학원, 대입조기선발반 모집... 12월 15일 개강","[용인비상에듀기숙학원 전경, 입시 전문 재수기숙학원 용인비상에듀기숙학원이 2021학..."


In [83]:
article_dataset.to_csv(path_or_buf = DATA_DIR + 'article_table.csv',index=False)

# vector화를 위한 데이터셋 재구성

### id와 ex_sentence 컬럼만있는 df

In [63]:
df = dataset.explode('ex_sentence').reset_index(drop=True)

In [64]:
df.head()

,id,title,sentence,extractive,ex_sentence
0,1,"논 타작물 재배, 2월 말까지 신청하세요","[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...","[2, 3, 10]",전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...
1,1,"논 타작물 재배, 2월 말까지 신청하세요","[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...","[2, 3, 10]",쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 ...
2,1,"논 타작물 재배, 2월 말까지 신청하세요","[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...","[2, 3, 10]",논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 ...
3,2,"여수시, 컬러빌리지 마무리...색채와 빛도시 완성","[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail...","[2, 4, 11]",여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...
4,2,"여수시, 컬러빌리지 마무리...색채와 빛도시 완성","[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail...","[2, 4, 11]",컬러빌리지는 색채를 이용한 경관개선사업으로 사업완료에 따라 고소천사벽화마을과 자산마...


In [69]:
df = df[['id', 'ex_sentence']]

In [70]:
df.head()

,id,ex_sentence
0,1,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...
1,1,쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 ...
2,1,논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 ...
3,2,여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...
4,2,컬러빌리지는 색채를 이용한 경관개선사업으로 사업완료에 따라 고소천사벽화마을과 자산마...


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731931 entries, 0 to 731930
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           731931 non-null  int64 
 1   ex_sentence  731931 non-null  object
dtypes: int64(1), object(1)
memory usage: 11.2+ MB


### for_vector_table 내보내기

In [72]:
df.to_csv(path_or_buf = DATA_DIR + 'for_vector_table.csv',index=False)